In [5]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
import os
import cv2
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout, BatchNormalization, Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Average

In [6]:
from google.colab import drive
drive.mount('/content/drive')

# Define base directory
base_dir = '/content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data'

# Verify each directory level
drive_dir = '/content/drive/MyDrive'
bootcamp_dir = os.path.join(drive_dir, 'BOOTCAMP')
colab_notebooks_dir = os.path.join(bootcamp_dir, 'ColabNotebooks')
project_dir = os.path.join(colab_notebooks_dir, 'ProjectWithGreg')
data_dir = os.path.join(project_dir, 'Data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Use os.path.join() to concatenate base_dir and 'train', set train_dir to 'Data/train'
# os.path.join(base_dir, 'train') takes base_dir, which = 'Data/', and concatenates it with string 'train'
train_dir = os.path.join(base_dir, 'train')

# Concatenate base_dir and 'test', set test_dir to 'Data/test'
# os.path.join(base_dir, 'test') takes base_dir, which = 'Data/', and concatenates it with string 'test'
test_dir = os.path.join(base_dir, 'test')

# Concatenate base_dir and 'valid', set valid_dir to 'Data/valid'
# os.path.join(base_dir, 'valid') takes base_dir, which = 'Data/', and concatenates it with string 'valid'
valid_dir = os.path.join(base_dir, 'valid')

#Read contents of base_dir directory and return list of names of entries (files and directories) in it
os.listdir(base_dir)

['valid', 'test', 'train', 'saved_model', 'best_model_resnet.keras']

In [10]:
# tf.keras.preprocessing.image_dataset_from_directory function generates tf.data.Dataset from image files in directory
# convenient way to load image data for training, validation, or testing in format that's easy to work with TensorFlow models

#FOR CHAINED MODEL, RESIZE IMAGES to 224, 224, 3 (what ResNet based model expects)

training_set = tf.keras.preprocessing.image_dataset_from_directory(     # image_dataset_from_directory method: images automatically labeled based on subdirectory names
                                                                        # each subdirectory treated as a class and labels assigned as integers starting from 0

train_dir,                  # Purpose: This is directory path where training images are stored
                            # Structure: should contain subdirectories, each representing different class; name of each subdirectory will be used as class label for images within it
seed=101,
image_size=(224, 224),
batch_size=32,
label_mode='int'           # to work with sparse labels, use 'int' as value for label_mode parameter
    )


testing_set = tf.keras.preprocessing.image_dataset_from_directory(
test_dir,                   # Purpose: This is directory path where test images are stored
                            # Structure: should contain subdirectories, each representing different class; name of each subdirectory will be used as class label for images within it
seed=101,
image_size=(224, 224),
batch_size=32,
label_mode='int'           # to work with sparse labels, use 'int' as value for label_mode parameter
    )


validation_set = tf.keras.preprocessing.image_dataset_from_directory(
valid_dir,                  # Purpose: This is directory path where valid images are stored
                            # Structure: should contain subdirectories, each representing different class; name of each subdirectory will be used as class label for images within it
seed=101,
image_size=(224, 224),
batch_size=32,
label_mode='int'           # to work with sparse labels, use 'int' as value for label_mode parameter
    )


Found 613 files belonging to 4 classes.
Found 315 files belonging to 4 classes.
Found 72 files belonging to 4 classes.


In [ ]:
#Alterations necessary for chaining both models
#(1) Resize images to 224, 224, 3 (what ResNet based model expects)
#(2) Remove input_shape from Data Augmentation definition
#(3) Remove all MaxPooling2D and Flatten layers from custom model
#(4) Remmove data augmentation and Rescaling from custom model and move them to complete (chained) model
#(5) Add BatchNormalization layer right after base model/at start of custom model to normalize 1D vector output before its fed into dense layers of custom model
#(6) Remove extra Dropout layers
#(7) Add input layer as first layer of chained model, specify input_shape there
#(8) Remove BatchNormalization layer from within custom_cnn
#(9) Insert ConverToTensorLayer between ResNet50 and custom_cnn, otherwise model can't be evaluated

In [11]:
#CHAIN PRE-TRAINED MODEL AND CUSTOM MODEL

#Specify img_size, channels, img_shape, and class_count before defining model and data pipeline
img_size = (224, 224)       #Resize to 224x224
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(training_set.class_names)

#class_count = len(list(training_set.class_indices.keys()))
#AttributeError: '_PrefetchDataset' object has no attribute 'class_indices' occurs because tf.data.Dataset objects, like _PrefetchDataset, don't have class_indices
#attribute; class_indices attribute is specific to ImageDataGenerator from Keras
#If using tf.data.Dataset objects created with image_dataset_from_directory, retrieve class information from class_names attribute
      #use class_names attribute from tf.data.Dataset to get class labels:
      #class_names = testing_set.class_names
      #class_count = len(class_names)
            # update code from class_indices to class_names
            # class_count = len(list(training_set.class_indices.keys())) becomes class_count = len(training_set.class_names)
            # If need to use class_count in model definitions or training configurations, ensure set correctly: class_count = len(training_set.class_names)

# Define data augmentation
data_augmentation = Sequential([
    RandomFlip("horizontal"), ##input_shape=(224, 224, 3)),      #Omit input_shape here, specify it instead in input layer of chained model
    RandomRotation(0.2),
    RandomZoom(0.2),
])

# Define pre-trained base model with ResNet50
# include_top=False removes top classification layer of ResNet50 model
# weights="imagenet" uses weights pre-trained on ImageNet dataset
# input_shape=img_shape sets input shape for model

# pooling='max' adds global max pooling layer at end of base model
# ResNet50 with include_top=False and pooling='max' produces 1D vector for output ----- Not true: It produced a list with shape =(None,2048)
# pooling='max' argument ensures output is single vector with 2048 features, compatible with dense layers directly
# output shape is 1D list of size 2048 (or number of features defined by base model)
# List must be converted to tensors before being passed to custom_cnn
# Insert conversion to tensor layer before DataNormalization layer

# Define conversion custom layer

# In Python, class definition should come before creating an instance of the class. Define the ConvertListToTensor class
# before creating an instance of it using custom_layer = ConvertListToTensor(name='my_custom_layer').

#Define custom layer class ConvertListToTensor:
class ConvertListToTensor(tf.keras.layers.Layer):     #defines new class 'ConvertListToTensor' that inherits from tf.keras.layers.Layer; class is blueprint or template for creating objects, defines properties (attributes) and behaviors (methods) that all objects of class will have
                                                      #means ConvertListToTensor is custom layer that can be used in TensorFlow Keras model

    def __init__(self, name=None):                              #constructor method for ConvertListToTensor class, initializes new instance of class; primary purpose of constructor is initialize newly created object by setting initial values for its attributes or performing any necessary setup tasks
                                                                #name=None parameter allows you to specify a name for layer when creating an instance

        super(ConvertListToTensor, self).__init__(name=name)    #calls constructor of parent class tf.keras.layers.Layer to initialize layer
                                                                #passes name parameter to parent class constructor

    def call(self, inputs):                                     #method defines behavior of layer when it is called during forward pass of model
                                                                #call method is required method in Keras layer

        tensor_data = tf.convert_to_tensor(inputs, dtype=tf.float32)    #converts input data (assumed to be list) back to TensorFlow tensor
                                                                        #tf.convert_to_tensor function converts input data to tensor with specified data type tf.float32

        return tensor_data                            #returns tensor data converted from input list back to calling function; calling function invokes or "calls"; called function is executed, calling function initiated the call

#Create instance of custom layer with specific name
#When specifying custom layer in model definition, refer to it by name assigned to layer instance (here, 'ConvertListToTensorLayer')

custom_layer = ConvertListToTensor(name='ConvertListToTensorLayer')


#Define ResNet50 base model

base_model = ResNet50(include_top=False, weights="imagenet", input_shape=img_shape, pooling='max')
for layer in base_model.layers:
    layer.trainable = False

#Custom Layers: When defining data augmentation layers or custom layers, ensure input shape matches shape of images

# Define custom CNN model
# MaxPooling2D Layers typically used to reduce spatial dimensions (height, width) of feature maps while retaining important information
#If using ResNet50 with pooling='max', output is already 1D vector (features extracted from entire image)
# additional MaxPooling2D layers unnecessary since feature extraction has already been done
# Flatten Layer used to convert 2D feature map into 1D vector, which is required to connect to dense layers
# When using pre-trained model with pooling='max', output is already 1D vector, so Flatten layer not needed

#Create new layer in between ResNet and Sequential models to handle data transformation without interrupting model evaluation process
#By incorporating data transformation as layer within model architecture, data flows seamlessly through entire neural network without explicit interruptions

#Class_count variable defines number of output classes for model; sets number of units in final Dense layer of model


# Define custom_CNN to be integrated with ResNet50
custom_cnn = Sequential([
    #Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),

    #MaxPooling2D(pool_size=(2, 2)),         #When using pooling='max', ResNet50 outputs 1D vector with shape (batch_size, 2048), where 2048 is features
                                             #extracted by model; need to flatten output if to add dense layers directly (but we're not adding dense layers yet)
                                             #If you add additional layers, directly connect to this 1D vector --- NOT TRUE: must convert list to tensors first

    #Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    #MaxPooling2D(pool_size=(2, 2)),
    #BatchNormalization(),                  #Add/move BatchNormalization layer to before start of custom_cnn or else model can't be evaluated
    #Dropout(0.25),
    #Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    #MaxPooling2D(pool_size=(2, 2)),
    #Dropout(0.25),
    #Flatten(),                              #Flatten layer only necessary if base model output is 2D feature map (grid of features); would then convert into 1D vector
    Dense(128, activation='relu'),           # Handle 1D input
    Dropout(0.25),
    Dense(class_count, activation='softmax')  # Output layer with number of classes
])


model = tf.keras.Sequential([
    # Add custom layer to the model

    # Add other layers as needed
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Create chained model by chaining custom CNN with ResNet50
# Data augmentation should be applied before rescaling, but both should be only in chained model (remove from custom)
# Custom Model Expectation: layers added after base model must correctly handle 1D vector; therefore, adding layers like Dense, Dropout, and BatchNormalization is appropriate
# You do not need Conv2D or Flatten layers after base model; they're for 2D feature maps, not 1D vectors
# Do need to put BatchNormalization layer in between base model and custom_cnn

model = Sequential([
    Input(shape=img_shape),   # Add Input layer as first layer, specify shape here; training threw errors without Input Layer
    data_augmentation,        # Apply data augmentation to chained model
    Rescaling(1./255),        # Apply rescaling to chained
    base_model,               # Base model: ResNet50
    ConvertListToTensor(name='ConvertListToTensorLayer'),     #BatchNormalization expecting tensors, not list
    BatchNormalization(),     # Add BatchNormalization layer because base model ouput incompatible with custom_cnn
    custom_cnn                # Custom CNN on top of base model
])

# Compile model
optimizer = Adam()
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#TensorFlow’s SavedModel format doesn’t use specific file extension; creates directory containing multiple files and subdirectories
#This format is TensorFlow’s default and is designed for better compatibility and integration with TensorFlow Serving

# Define filepath to save best model
# to save model to specific directory, provide complete absolute path
# Define base directory path if necessary
#base_dir = '/content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data'

# Create base directory if it doesn't exist
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Define full file path including base directory
filepath = os.path.join(base_dir, 'best_model_resnet.keras')

model.save(filepath)  # Saves in specified directory

# Create ModelCheckpoint callback to save best model based on validation accuracy
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')



#Input 0 of layer "functional" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(None, 2048)
#model receiving tensor with shape (None, 2048) when it expects tensor shape (None, 224, 224, 3); issue with how layers are connected, particularly after base model (ResNet50)
#pre-trained model with include_top=False output will be 2D tensor (e.g., (None, 7, 7, 2048)) if you don't specify pooling, or 1D tensor (e.g., (None, 2048)) if you use global pooling

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
# Train chained model with added callback
history = model.fit(
    x=training_set,
    epochs=50,
    verbose=1,
    validation_data=validation_set,
    callbacks=[checkpoint]
)

In [12]:
#Prepare to load saved model

# Import necessary libraries
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.models import load_model

#Mount Google Drive:
from google.colab import drive
drive.mount('/content/drive')

# Define directory path
directory_path = '/content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data/'

# List contents of directory
!ls -l {directory_path}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 93859
-rw------- 1 root root 96094261 Aug  8 23:27 best_model_resnet.keras
drwx------ 2 root root     4096 Jul 31 17:32 saved_model
drwx------ 6 root root     4096 Jul 15 23:58 test
drwx------ 6 root root     4096 Jul 15 23:58 train
drwx------ 6 root root     4096 Jul 15 23:58 valid


there is an issue with deserializing the model due to a custom class named 'ConvertListToTensor' that is not being recognized during the deserialization process. The error message indicates that the class 'ConvertListToTensor' needs to be properly registered for deserialization.

To resolve this issue, you can follow these steps:

Register the custom class 'ConvertListToTensor' with Keras deserialization by using the @keras.saving.register_keras_serializable() decorator.

Define a from_config method in the custom class to properly deserialize the object.

Here is an example of how you can modify your custom class 'ConvertListToTensor' to ensure it can be properly deserialized:



---Load the model after registering the custom class
model = load_model(model_path, custom_objects={'ConvertListToTensor': ConvertListToTensor})
By registering the custom class 'ConvertListToTensor' with the @keras.saving.register_keras_serializable() decorator and providing it as a custom object when loading the model, you should be able to deserialize the model successfully.

To ensure that components that are Python object instances (layers, models, etc.) returned by get_config() are explicitly deserialized in the model's from_config() method, you need to make sure that any custom layers or classes used in your model are properly registered for serialization and deserialization.

Here are the steps to address this issue:

Register Custom Layers for Serialization: If your model contains custom layers or classes, you need to ensure that these custom components are properly registered for serialization and deserialization. You can use the @keras.saving.register_keras_serializable() decorator to register your custom layers. This decorator ensures that the custom classes are recognized during the serialization and deserialization process.

Implement get_config() and from_config() Methods: For any custom layers or classes that you have defined, make sure they implement the get_config() method to return the configuration of the object and the from_config() method to recreate the object from its configuration. These methods are essential for proper serialization and deserialization of custom components.

Provide Custom Objects when Loading the Model: When loading the model using load_model, make sure to provide any necessary custom objects using the custom_objects parameter. This allows TensorFlow to correctly deserialize the custom components used in the model.

How you can register a custom layer and ensure it can be properly deserialized:  



-- Load the model and provide the custom object
model = load_model(model_path, custom_objects={'CustomLayer': CustomLayer})
By following these steps and ensuring that custom components are properly registered and implement the necessary methods for serialization and deserialization, you should be able to resolve the error related to deserialization issues in TensorFlow/Keras models.

In [17]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from tensorflow.keras.layers import deserialize as keras_deserialize
from tensorflow.keras.models import load_model, save_model

@keras.saving.register_keras_serializable()
class CustomLayer(Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)

    def call(self, inputs):
        return K.dot(inputs, self.w)

    def get_config(self):
        config = super(ConvertToTensorLayer, self).get_config()
        config.update({'output_dim': self.output_dim})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

# Specify correct path to model
model_path = os.path.join(directory_path, 'best_model_resnet.keras')
model = load_model(model_path, custom_objects={'ConvertListToTensor': ConvertListToTensor})

# Save the model
#model.save('my_model.h5')


AttributeError: module 'tensorflow.keras' has no attribute 'saving'

In [15]:
from tensorflow.keras.models import load_model

# Specify correct path to model
#model_path = os.path.join(directory_path, 'best_model_resnet.keras')
#model = load_model(model_path)

# Specify correct path to model
model_path = os.path.join(directory_path, 'best_model_resnet.keras')
model = load_model(model_path, custom_objects={'ConvertListToTensor': ConvertListToTensor})

#By using tensorflow.keras.models.load_model, you should be able to load the model successfully
#Make sure to provide the custom object ConvertListToTensor as before if it is needed for deserialization of the model





TypeError: <class 'keras.src.models.sequential.Sequential'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras', 'class_name': 'Sequential', 'config': {'name': 'sequential_3', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None, 'shared_object_id': 134103265646176}, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 224, 224, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_1'}, 'registered_name': None}, {'module': 'keras', 'class_name': 'Sequential', 'config': {'name': 'sequential', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None, 'shared_object_id': 134103265646176}, 'layers'

In [14]:
from keras.layers import Layer
from keras import backend as K
from keras.layers import deserialize as keras_deserialize

@keras.saving.register_keras_serializable()
class ConvertListToTensor(Layer):
    def __init__(self, name='ConvertListToTensorLayer', **kwargs):
        super(ConvertListToTensor, self).__init__(name=name, **kwargs)

    def call(self, inputs):
        return K.constant(inputs)

    def get_config(self):
        config = super(ConvertListToTensor, self).get_config()
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

AttributeError: module 'tensorflow.keras' has no attribute 'saving'

In [ ]:
# calculate ts_length by multiplying length of testing_set (len(testing_set)) by batch size
# Store this calculation in variable 'ts_length'
ts_length = len(testing_set) * 32  # Adjust based on actual batch size

# Determine batch size and steps for evaluation
test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length % n == 0 and ts_length / n <= 80]))
test_steps = ts_length // test_batch_size

In [ ]:
# Evaluate model
train_score = model.evaluate(training_set, steps=test_steps, verbose=1)
valid_score = model.evaluate(validation_set, steps=test_steps, verbose=1)
test_score = model.evaluate(testing_set, steps=test_steps, verbose=1)

# Print evaluation results
print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

AttributeError: Exception encountered when calling BatchNormalization.call().

[1m'list' object has no attribute 'shape'[0m

Arguments received by BatchNormalization.call():
  • inputs=['tf.Tensor(shape=(None, 2048), dtype=float32)']
  • training=False
  • mask=['None']

In [ ]:
import tensorflow as tf

training_data_tensor = tf.convert_to_tensor(training_set, dtype=tf.float32)
testing_data_tensor = tf.convert_to_tensor(testing_set, dtype=tf.float32)
validation_data_tensor = tf.convert_to_tensor(validation_set, dtype=tf.float32)

#Evaluate model using converted tensor
#score = model.evaluate(data_tensor, steps=test_steps, verbose=1)


# Evaluate model on training set
training_score = model.evaluate(training_set, steps=test_steps, verbose=1)
print("Training Set Evaluation:")
print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)

# Evaluate model on testing set
testing_score = model.evaluate(testing_set, steps=test_steps, verbose=1)
print("Testing Set Evaluation:")
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])
print('-' * 20)

# Evaluate model on validation set
validation_score = model.evaluate(validation_set, steps=test_steps, verbose=1)
print("Validation Set Evaluation:")
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)

ValueError: Attempt to convert a value (<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>) with an unsupported type (<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>) to a Tensor.

In [ ]:
#If you did not use a data image generator to create the training, testing, and validation sets of data, and you have the entire datasets loaded into memory,
#it is not necessary to use generators for evaluation. Using generators is beneficial when dealing with large datasets that cannot fit into memory all at once,
#as generators allow you to load data in batches during training and evaluation. When you have the entire dataset in memory, you can directly evaluate the model
#on the full dataset without the need for generators. Therefore, if you have already loaded the training, testing, and validation data into memory and do not need
#to process the data in batches, you can use the first method for evaluating the model (rather than the one below)

#ts_length = len(test_df)
#test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
#test_steps = ts_length // test_batch_size
#train_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
#valid_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)
#test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

#print("Train Loss: ", train_score[0])
#print("Train Accuracy: ", train_score[1])
#print('-' * 20)
#print("Validation Loss: ", valid_score[0])
#print("Validation Accuracy: ", valid_score[1])
#print('-' * 20)
#print("Test Loss: ", test_score[0])
#print("Test Accuracy: ", test_score[1])

SyntaxError: invalid syntax (<ipython-input-15-cfd9e07a233d>, line 3)


Data augmentation and rescaling typically occur before the data is fed into the base_model or any other parts of the neural network.  
  
Data Augmentation and Rescaling: These preprocessing steps are essential to ensure that the input data is properly formatted and normalized before it reaches the model. Augmentation helps in generalizing the model by artificially increasing the diversity of the training data, and rescaling normalizes the pixel values, which helps in stabilizing and speeding up the training process.
  
Location in the Pipeline: These steps are applied to the images as they are being loaded and processed, which means they happen before the images are passed to the base_model. The purpose is to ensure that the images are in the right format and scale when they enter the model.  
  
Illustration of process:  
  
Image Loading: Images are loaded from the directory.  
Data Augmentation and Rescaling: Applied to the images as they are being loaded.  
Model Processing: The preprocessed images are then fed into the base_model and subsequently through the custom layers added on top.  

In a model where you are chaining a custom architecture with a pre-trained model like ResNet50, you need to place the data augmentation and rescaling layers appropriately. Here’s how you should approach it:

Steps to Chain a Custom Model with ResNet50
Data Augmentation: Data augmentation should be applied to the input images before they are fed into the base model (ResNet50). This preprocessing helps to increase the diversity of the training data by applying transformations like rotations, flips, etc.

Rescaling: Rescaling should be applied after data augmentation. This is because the Rescaling layer adjusts the pixel values to the appropriate range expected by the model. For ResNet50, this is usually between 0 and 1.

UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  warning indicates that you should use an Input layer as the first layer in your Sequential model, rather than specifying input_shape directly in the first layer. This is the preferred method in TensorFlow/Keras when building Sequential models

  You specified input_shape directly in the RandomFlip layer, which is part of the data augmentation process. This can lead to the warning you received. To address this, you should use an Input layer as the first layer in your Sequential model to define the input shape, and remove the input_shape argument from other layers.


It seems like there is an issue with the BatchNormalization layer in your model. The error message indicates that the BatchNormalization layer is receiving a list instead of a tensor with a specific shape.

To resolve this issue, you may need to check the input data that is being passed to the BatchNormalization layer. The BatchNormalization layer expects a tensor as input, but it seems to be receiving a list instead.

You can try to debug this issue by checking the shape of the data being passed to the BatchNormalization layer. Make sure that the data is in the correct format and shape that the BatchNormalization layer expects.

If you continue to encounter issues, you may need to review the implementation of your model and ensure that the data is being processed correctly before reaching the BatchNormalization layer.

ValueError: Exception encountered when calling Sequential.call().  
Input 0 of layer "functional_2" is incompatible with the layer: expected shape=(None, 200, 200, 3), found shape=(None, 2048)  

ValueError you're encountering suggests that there is a shape mismatch between what your model expects and what it's receiving. Specifically, the model is expecting an input shape of (None, 200, 200, 3) but is receiving input with the shape (None, 2048)  

Here are some potential causes and solutions for this issue:

1. Mismatch Between Feature Extraction and Custom Layers  
This error commonly arises when the output of a base model (like ResNet50) is incompatible with the input shape expected by the subsequent layers in the Sequential model. In your case, it seems like the feature extraction part (e.g., ResNet50) is outputting a tensor of shape (None, 2048), which is not being properly connected to the subsequent layers in your Sequential model.

2. Base Model Output Shape  
If you're using a pre-trained model like ResNet50, it usually outputs feature maps of a different shape than expected for a fully connected layer. The feature extraction layer's output needs to be appropriately processed before feeding it into subsequent layers.

Solutions  

Solution 2: Ensure Correct Shape Handling  
Make sure that you correctly handle the shape output by the base model. For instance, if you use pooling='max' in ResNet50, its output will be a 1D vector (2048 features), which is expected as input to a dense layer.  
  
Solution 3: Adjust Input Size  
Ensure the input size in your data generator matches the expected input size of your model. If you decide to use (224, 224) instead of (200, 200), ensure all parts of the model and data pipelines are consistent.

The error you are encountering, AttributeError: 'list' object has no attribute 'shape', is likely due to the way the dataset is being passed to the model for evaluation. The BatchNormalization layer expects a tensor as input, but it seems that a list is being passed instead.

In the context of evaluating the model, the issue arises because the BatchNormalization layer expects a tensor as input, but it is receiving a list instead. During training, the model may be able to handle the list output from ResNet without issues because the training process involves backpropagation and gradient descent, which can accommodate certain data formats.

However, during evaluation, the model is expected to make predictions on new data without updating its weights. This prediction process requires the data to be in the correct tensor format for the layers to process the information correctly.

When the BatchNormalization layer encounters a list instead of a tensor during evaluation, it raises an error because it cannot process the data in that format. This discrepancy in data format between training and evaluation can lead to errors specifically related to the BatchNormalization layer, which requires tensor inputs to perform normalization operations.

Therefore, to ensure successful evaluation of the model, it is crucial to pass the evaluation datasets in the correct tensor format, allowing each layer, including BatchNormalization, to process the data appropriately and make predictions accurately.

BatchNormalization is a layer in neural networks that normalizes the inputs to a layer for each mini-batch. This helps in stabilizing the learning process and dramatically reduces the number of training epochs required to train deep networks.  
  
What Batch Normalization Does:  
Normalization: It normalizes the output of the previous activation layer by subtracting the batch mean and dividing by the batch standard deviation. This centers the data around zero with a standard deviation of one.

Scale and Shift: After normalization, it applies a scaling factor (gamma) and a shift factor (beta), which allows the layer to learn the optimal scale and mean of the inputs. This ensures the layer can represent the identity transformation if necessary.  
  
Why Use Batch Normalization:  
Speed Up Training: By normalizing the inputs, it helps in stabilizing the learning process. This often allows for higher learning rates, leading to faster convergence.  
  
Regularization: It has a slight regularizing effect, reducing the need for other forms of regularization like dropout. This happens because the mini-batch statistics add some noise to each training step.  
  
Reduce Internal Covariate Shift: It mitigates the problem of internal covariate shift, where the distribution of each layer's inputs changes during training. By maintaining the inputs to each layer with a more consistent distribution, it simplifies and accelerates training.  
  
Gradient Propagation: It helps in maintaining gradients in a range that prevents them from vanishing or exploding, making it easier to train deeper networks.  
  
Example of Batch Normalization in the Model:
In your case, BatchNormalization is used after the output of the base model and before the dense layers. This helps in ensuring that the inputs to the dense layers are normalized, improving the training process.










The error message you are encountering indicates that there is an issue with the type of data being passed to the model.evaluate() method. The model.evaluate() method typically expects tensors or numpy arrays as input data, but in this case, it seems like you are passing a dataset object (_PrefetchDataset) instead.

To resolve this issue, you need to extract the data from your dataset object before passing it to the model.evaluate() method. You can do this by iterating over the dataset and converting it to tensors or numpy arrays that can be used for evaluation.

CONVERT DATA BACK INTO TENSORS  

If the data is in tensor format when it enters the first model (ResNet) and then gets converted to a list format before being passed to the subsequent layers in the model chain, you can convert the data back into tensors within the model architecture itself. One way to achieve this is by using a custom layer that performs the conversion from lists to tensors.  